In this homework, we will use the Car price dataset. Download it from

https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge


from IPython.display import display

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

Features

For the rest of the homework, you'll need to use only these columns:

    Make,
    Model,
    Year,
    Engine HP,
    Engine Cylinders,
    Transmission Type,
    Vehicle Style,
    highway MPG,
    city mpg


In [2]:
features = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]

Data preparation
Select only the features from above and transform their names using next line:

data.columns = data.columns.str.replace(' ', '_').str.lower()

Fill in the missing values of the selected features with 0.

Rename MSRP variable to price.

In [3]:
df = pd.read_csv('data.csv')
df = df[ features ]
df.columns = df.columns.str.replace(' ', '_').str.lower()
df[ 'price' ] = df[ 'msrp' ]
del df[ 'msrp' ]
df = df.fillna(0)
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
price                0
dtype: int64

Question 1

What is the most frequent observation (mode) for the column transmission_type?

    AUTOMATIC
    MANUAL
    AUTOMATED_MANUAL
    DIRECT_DRIVE

Answer: AUTOMATIC -> 8266


In [4]:
df['transmission_type'].value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

    engine_hp and year
    engine_hp and engine_cylinders
    highway_mpg and engine_cylinders
    highway_mpg and city_mpg

Answer: highway_mpg and city_mpg

In [5]:
columns_subset = [ 'engine_hp', 'year', 'engine_cylinders', 'highway_mpg', 'city_mpg' ]
df[ columns_subset ].corr()

,engine_hp,year,engine_cylinders,highway_mpg,city_mpg
engine_hp,1.000000,0.338714,0.774851,-0.415707,-0.424918
year,0.338714,1.000000,-0.040708,0.258240,0.198171
engine_cylinders,0.774851,-0.040708,1.000000,-0.614541,-0.587306
highway_mpg,-0.415707,0.258240,-0.614541,1.000000,0.886829
city_mpg,-0.424918,0.198171,-0.587306,0.886829,1.000000


Make price binary

    Now we need to turn the price variable from numeric into a binary format.
    Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.


In [6]:
priceMean = df['price'].mean()
print( "priceMean = ", priceMean)
df[ 'above_average' ] = (df['price'] > priceMean ) + 0
#df[ 'age' ] = 2023 - df[ 'year' ]
#del df[ 'year' ]
cleaned_df = df.copy()
display( df )

priceMean =  40594.737032063116


,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0
...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120,1
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670,1
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620,1
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920,1


Split the data

    Split your data in train/val/test sets with 60%/20%/20% distribution.
    Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
    Make sure that the target value (above_average) is not in your dataframe.


In [7]:
seed = 42
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=seed)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=seed)

y_train = df_train[ 'above_average' ]
y_val = df_val[ 'above_average' ]

del df_train[ 'above_average' ]
del df_val[ 'above_average' ]

Question 3

    Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.
    Round the scores to 2 decimals using round(score, 2).

Which of these variables has the lowest mutual information score?

    make
    model
    transmission_type
    vehicle_style

Answer: transmission_type


In [8]:
categorical_columns = [ 'make' , 'model', 'transmission_type', 'vehicle_style']
numerical_columns = [ 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year' ]

def calculate_mi(series):
    return mutual_info_score(series, y_train)

df_mi = df_train[categorical_columns].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

df_mi.MI = round( df_mi.MI, 2 )

#display(df_mi.head())
#display(df_mi.tail())
display(df_mi)

,MI
model,0.46
make,0.24
vehicle_style,0.08
transmission_type,0.02


Question 4

    Now let's train a logistic regression.
    Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
    Fit the model on the training dataset.
        To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
        model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

    0.60
    0.72
    0.84
    0.95


In [9]:
train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
train_dict[0]

{'make': 'Mitsubishi',
 'model': 'Endeavor',
 'transmission_type': 'AUTOMATIC',
 'vehicle_style': '4dr SUV',
 'engine_hp': 225.0,
 'engine_cylinders': 6.0,
 'highway_mpg': 19,
 'city_mpg': 15,
 'year': 2011}

Include categorical variables using one-hot encoding.

In [10]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)

X_train.shape

(7148, 943)

In [11]:
#dv.get_feature_names()
dv.get_feature_names_out()

array(['city_mpg', 'engine_cylinders', 'engine_hp', 'highway_mpg',
       'make=Acura', 'make=Alfa Romeo', 'make=Aston Martin', 'make=Audi',
       'make=BMW', 'make=Bentley', 'make=Bugatti', 'make=Buick',
       'make=Cadillac', 'make=Chevrolet', 'make=Chrysler', 'make=Dodge',
       'make=FIAT', 'make=Ferrari', 'make=Ford', 'make=GMC',
       'make=Genesis', 'make=HUMMER', 'make=Honda', 'make=Hyundai',
       'make=Infiniti', 'make=Kia', 'make=Lamborghini', 'make=Land Rover',
       'make=Lexus', 'make=Lincoln', 'make=Lotus', 'make=Maserati',
       'make=Maybach', 'make=Mazda', 'make=McLaren', 'make=Mercedes-Benz',
       'make=Mitsubishi', 'make=Nissan', 'make=Oldsmobile',
       'make=Plymouth', 'make=Pontiac', 'make=Porsche',
       'make=Rolls-Royce', 'make=Saab', 'make=Scion', 'make=Spyker',
       'make=Subaru', 'make=Suzuki', 'make=Tesla', 'make=Toyota',
       'make=Volkswagen', 'make=Volvo', 'model=1 Series', 'model=100',
       'model=124 Spider', 'model=190-Class', 'model

Fit the model on the training dataset.

In [12]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [13]:
val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [14]:
model.predict_proba(X_val)

array([[0.9912654 , 0.0087346 ],
       [0.00966721, 0.99033279],
       [0.99842511, 0.00157489],
       ...,
       [0.99854025, 0.00145975],
       [0.02821712, 0.97178288],
       [0.03751833, 0.96248167]])

In [15]:
y_val_pred_proba = model.predict_proba(X_val)[:, 1]
y_val_pred_direct = model.predict(X_val)

In [16]:
y_val_pred = (y_val_pred_proba > 0.46)
display( y_val_pred )

array([False,  True, False, ..., False,  True,  True])

In [17]:
val_accuracy_proba = (y_val_pred == y_val).mean()
original_accuracy_proba = val_accuracy_proba
val_accuracy_proba = round( val_accuracy_proba, 2 )

print( "val_accuracy_proba = ", val_accuracy_proba )

val_accuracy_proba =  0.94


In [18]:
val_accuracy_direct = (y_val_pred_direct == y_val).mean()
original_accuracy_direct = val_accuracy_direct
val_accuracy_direct = round( val_accuracy_direct, 2 )

print( "val_accuracy_direct = ", val_accuracy_direct )

val_accuracy_direct =  0.93


Question 5

    Let's find the least useful feature using the feature elimination technique.
    Train a model with all these features (using the same parameters as in Q4).
    Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
    For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

    year
    engine_hp
    transmission_type
    city_mpg

Answer: engine_hp

In [19]:
removed_column =[ 'year', 'engine_hp', 'transmission_type', 'city_mpg']
categorical_columns = [ 'make' , 'model', 'transmission_type', 'vehicle_style']
numerical_columns = [ 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year' ]

categorical_columns_variant = [ 
                [ 'make' , 'model', 'transmission_type', 'vehicle_style'],
                [ 'make' , 'model', 'transmission_type', 'vehicle_style'],
                [ 'make' , 'model',  'vehicle_style'],
                [ 'make' , 'model', 'transmission_type', 'vehicle_style']                           
]

numerical_columns_variant = [ 
                [ 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg' ],
                [ 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year' ],
                [ 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year' ],
                [ 'engine_hp', 'engine_cylinders', 'highway_mpg', 'year' ]
]

for i in range(len(categorical_columns_variant)):
    crt_removed_column = removed_column[ i ]
    crt_categorical_column = categorical_columns_variant[i]
    crt_numerical_column = numerical_columns_variant[i]
    #print( crt_categorical_column )
    #print( crt_numerical_column )

    crt_df = cleaned_df.copy()
    crt_df = crt_df[ crt_categorical_column + crt_numerical_column + ['price']]
    crt_df[ 'above_average' ] = (crt_df['price'] > priceMean ) + 0
    seed = 42
    crt_df_train_full, crt_df_test = train_test_split(crt_df, test_size=0.2, random_state=seed)
    crt_df_train, crt_df_val = train_test_split(crt_df_train_full, test_size=0.25, random_state=seed)


    crt_y_train = crt_df_train[ 'above_average' ]
    crt_y_val = crt_df_val[ 'above_average' ]

    del crt_df_train[ 'above_average' ]
    del crt_df_val[ 'above_average' ]

    crt_train_dict = crt_df_train[crt_categorical_column + crt_numerical_column].to_dict(orient='records')
    crt_val_dict = crt_df_val[crt_categorical_column + crt_numerical_column].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    dv.fit(crt_train_dict)

    crt_X_train = dv.transform(crt_train_dict)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(crt_X_train, crt_y_train)

    crt_X_val = dv.transform(crt_val_dict)

    y_val_pred_proba = model.predict_proba(crt_X_val)[:, 1]
    y_val_pred_direct = model.predict(crt_X_val)

    y_val_pred = (y_val_pred_proba > 0.46)
    #display( y_val_pred )

    val_accuracy_proba = (y_val_pred == crt_y_val).mean()
    val_accuracy_proba = round( val_accuracy_proba, 2 )

    val_accuracy_direct_unrounded = (y_val_pred_direct == y_val).mean()
    val_accuracy_direct = round( val_accuracy_direct, 2 )

    accuracy_delta = original_accuracy_direct - val_accuracy_direct_unrounded
    
    #print( "crt_removed_column = ", crt_removed_column, "==> val_accuracy_proba = ", val_accuracy_proba, " val_accuracy_direct = ", val_accuracy_direct )
    print( "crt_removed_column = ", crt_removed_column, "==> accuracy_delta = ", accuracy_delta )
    
    #display( crt_df.head() )

crt_removed_column =  year ==> accuracy_delta =  -0.013848090642047839
crt_removed_column =  engine_hp ==> accuracy_delta =  -0.00041963911036513313
crt_removed_column =  transmission_type ==> accuracy_delta =  -0.010490977759127218
crt_removed_column =  city_mpg ==> accuracy_delta =  -0.011330255979857373


Question 6

    For this question, we'll see how to use a linear regression model from Scikit-Learn.
    We'll need to use the original column price. Apply the logarithmic transformation to this column.
    Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
    This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
    Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

    0
    0.01
    0.1
    1
    10

    Note: If there are multiple options, select the smallest alpha.


In [21]:
alpha_vals = [0, 0.01, 0.1, 1, 10]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

for alpha in alpha_vals:
    print( "alpha = ", alpha )

    crt_df = cleaned_df.copy()
    crt_df = crt_df[ categorical_columns + numerical_columns + ['price']]
    crt_df['log_price'] = np.log1p(crt_df['price'])

    seed = 42
    crt_df_train_full, crt_df_test = train_test_split(crt_df, test_size=0.2, random_state=seed)
    crt_df_train, crt_df_val = train_test_split(crt_df_train_full, test_size=0.25, random_state=seed)

    crt_y_train = crt_df_train[ 'log_price' ]
    crt_y_val = crt_df_val[ 'log_price' ]

    del crt_df_train[ 'log_price' ]
    del crt_df_val[ 'log_price' ]

    del crt_df_train[ 'price' ]
    del crt_df_val[ 'price' ]

    crt_train_dict = crt_df_train[crt_categorical_column + crt_numerical_column].to_dict(orient='records')
    crt_val_dict = crt_df_val[crt_categorical_column + crt_numerical_column].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    dv.fit(crt_train_dict)

    crt_X_train = dv.transform(crt_train_dict)
    crt_X_val = dv.transform(crt_val_dict)

    
    ridgeModel = Ridge(alpha = alpha, solver = 'sag')
    ridgeModel.fit(crt_X_train, crt_y_train)
    #train_score = ridgeModel.score(crt_X_train, crt_y_train)
    y_train_pred = model.predict(crt_X_train)
    train_rmse = round( rmse( crt_y_train, y_train_pred), 3 )

    #val_score = ridgeModel.score(crt_X_val, crt_y_val)
    y_val_pred = model.predict(crt_X_val)
    val_rmse = round( rmse( crt_y_val, y_val_pred), 3 )

    print( "val_rmse = ")
    display( val_rmse )

    #print( "alpha = ", alpha, " ==>train_rmse = " + train_rmse + " val_rmse =" + val_rmse )


alpha =  0
val_rmse = 


/home/midnightmover/machine-learning-zoomcamp/mlzoomcap-notebooks/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


9.868

alpha =  0.01
val_rmse = 


/home/midnightmover/machine-learning-zoomcamp/mlzoomcap-notebooks/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

/home/midnightmover/machine-learning-zoomcamp/mlzoomcap-notebooks/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


9.868

alpha =  10
val_rmse = 


/home/midnightmover/machine-learning-zoomcamp/mlzoomcap-notebooks/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


9.868